# In case of using colab
Remember to connect to your drive and upload the dataset if you want to repeat several times  this tutorial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
!ls

# Download the dataset

In this occasion we are going to use different video footage of cherry trees to test a tracking algorithm.


[videos-cherry](https://drive.google.com/drive/folders/1H0jr8uhQLkxs_wNjQqUiews_CLIFUdMV?usp=sharing)


# Install repositories
The repository for this hands on is:

```code
git clone https://github.com/LuisCossioUOH/Hands-On-Lacoro2025-Fruit-Tracking
```

You can either clone the reposritory or upload the tracking_utils.py file:

And install ultralytics
```code
pip install ultralytics==8.3.94
```

In [ ]:
# !git clone https://github.com/LuisCossioUOH/Hands-On-Lacoro2025-Fruit-Tracking
# !mv Hands-On-Lacoro2025-Fruit-Tracking/tracking_utils.py  ./tracking_utils.py
!pip install ultralytics==8.3.94

## Import relevant libraries

In [ ]:
import os
import cv2
import yaml

from ultralytics import YOLO

import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from matplotlib.patches import Rectangle


# Track dataset Dummy
1. Test the tracking dataset and check the difference between tracks and detections.
2. Draw the detections in the image.
3. Draw the tracks of each object with a diferent color.

In [ ]:
from tracking_utils import TrackingEllipseGenerator

dataset = TrackingEllipseGenerator(300,n_tracks=50,image_shape=[300,300])



In [ ]:
frame_id = 10
tracks,img = dataset.get_frame(frame_id)
plt.imshow(img)



In [ ]:
# frame_id = 100
frame_id += 1
tracks, image, dets = dataset.get_frame_and_detections(frame_id)

color = (0,255,0)
thickness = 2
print("dets: ",dets)
point1 = dets[0,:2].astype(np.int32)
point2 = dets[0,2:4].astype(np.int32)
image = cv2.rectangle(image, point1, point2, color=color, thickness=thickness)

#### CODE HERE ####

plt.imshow(image)

# Tracking a single object
We are going to see if we can can build a tracker for a single object.
1. Analize multiple frames and determine main challenges.

In [ ]:
dataset = TrackingEllipseGenerator(300,n_tracks=1,image_shape=[300,300],duration_samples=[299,300],
                                   false_negative_rate=0,false_positive_rate=0.5)

idx = 0

In [ ]:
idx += 1
tracks,img,dets = dataset.get_frame_and_detections(idx,draw_detection=True)
plt.imshow(img)

# Kalman tracking
1. Define a function calculate the L2 distance between a vector of size [4] and a matrix of size [n,4]. And produce n distances.
2. Modified the Tracker class to have a find_observation method. This recieves N new detections and return the closest to the currently tracked object. If there are no close detections then return and empty array.
3. Modified to code to have an input "u" diferent than 0. The input "u" represent the acceleration.

4. Modified the code to use the prior as the final track and increase the variance by 50% in the covariance matrix if there are no detections.



In [ ]:
class KalmanFilter:
    def __init__(self, A: np.ndarray, B: np.ndarray, C: np.ndarray, D: np.ndarray, cov_model_diag=1.0,
                 cov_obs_diag=1.0):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.Q = np.eye(len(A)) * np.array([cov_model_diag])
        self.R = np.eye(len(C)) * np.array([cov_obs_diag])

    def prior(self, x, u):
        return self.A @ x + self.B @ u

    def observation(self, x, u):
        return self.C @ x + self.D @ u

    def posterior(self, x, prior_x, u, observation, P):
        z_prior = self.observation(x, u)  # estimation
        prior_P = self.A @ P @ self.A + self.Q
        W = prior_P @ self.C.transpose() @ np.linalg.inv(self.C @ prior_P @ self.C.transpose() + self.R)  # Kalman gain
        post_x = prior_x + W @ (observation - z_prior)  # posterior
        post_P = (np.eye(len(P)) - W @ self.C) @ prior_P  #
        return post_x, post_P

def get_kalman_filter(variance_model= 20,variance_observation=40):
    A = np.array([[1, 0, 0, 0, 1, 0, 0, 0],
                  [0, 1, 0, 0, 0, 1, 0, 0],
                  [0, 0, 1, 0, 0, 0, 1, 0],
                  [0, 0, 0, 1, 0, 0, 0, 1],
                  [0, 0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 1]])

    B = np.array([[0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    C = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 1]])

    D = np.array([[0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0]])

    return KalmanFilter(A, B, C, D,variance_model, variance_observation)


def format_detections(detections: np.ndarray, speed=0):
    """
    Function to format detections into a position format
    :param detections: detection array to be formatted into a matrix of columns [x1,y1,x2,y2,x'1,y'1,x'2,y'2]. Initial
    speed is zero.
    :return:
    """
    detection = np.zeros([len(detections), 8])
    detection[:, :4] = detections[:, :4]
    detection[:, 4:] += speed
    return detection

kalman = get_kalman_filter()

In [ ]:
def calculate_iou_distance(target: np.ndarray, matrix2: np.ndarray):
    """
    Calculate IoU correlation between locations in matrices
    :param target: Position matrix array as [x1,y1,x2,y2] columns
    :param matrix2: Position matrix array as [x1,y1,x2,y2] columns
    :return:
    """
    target = np.reshape(target,[-1,4])
    inter = np.maximum(np.minimum(target[:, None, 2], matrix2[:, 2]) - np.maximum(target[:, None, 0], matrix2[:, 0]),
                       0)
    inter *= np.maximum(np.minimum(target[:, None, 3], matrix2[:, 3]) - np.maximum(target[:, None, 1], matrix2[:, 1]),
                        0)
    eps = 0.000001

    # Union Area
    w1, h1 = target[:, 2] - target[:, 0], target[:, 3] - target[:, 1] + eps
    w2, h2 = matrix2[:, 2] - matrix2[:, 0], matrix2[:, 3] - matrix2[:, 1] + eps
    union = ((w1 * h1)[:, None] + w2 * h2) - inter + eps

    iou = inter / union
    return iou

def calculate_distance(target: np.ndarray, matrix2: np.ndarray):
    """
    Calculate the distance between the target vector and some detection matrix
    :param target: Position matrix array as [x1,y1,x2,y2] columns
    :param matrix2: Position matrix array as [x1,y1,x2,y2] columns
    :return:
    """
    n_rows = matrix2.shape[0]
    distance = np.zeros([n_rows])
    for i in range(n_rows):
        distance[i] = np.linalg.norm(target - matrix2[i,:])
    return distance

class Tracker:
    def __init__(self,initial_position,kalman_filter:KalmanFilter,initial_covariance = 10):
        self.target = initial_position
        self.tracks = [initial_position.copy()]
        self.kalman = kalman_filter
        self.covariance_target = np.eye(8) * initial_covariance

    def find_observation(self,new_detections, threshold=80):
        ###### CODE HERE ######
        pass


        # return format_detections(new_detections[index_lower][None])

    def process_frame(self,new_detections):

        observation = self.find_observation(new_detections)

        if len(new_detections) == 0:
            u = np.zeros([4])
        # else:
        ###### CODE HERE ######

        prior_location = self.kalman.prior(self.target,u)

        if len(observation) > 0:

            # observation = format_detections(observation[:4])
            observation = observation[0,:]
            posterior_location, updated_covariance =  self.kalman.posterior(x=self.target,
                                                                                    prior_x = prior_location,
                                                                                    u = np.zeros([4]),
                                                                                    observation = observation,
                                                                                    P = self.covariance_target)
            self.covariance_target = updated_covariance
            self.target = posterior_location
            self.tracks.append(posterior_location.copy())




    def plot_tracking_position(self,img,color=(0,0.6,0.05)):

        fig, ax = plt.subplots()
        p1 = self.target[:2]
        wh = self.target[2:4] - self.target[:2]
        ax.add_patch(Rectangle(p1.tolist(), wh[0], wh[1],
                               edgecolor = 'red',
                               facecolor = 'blue',
                               fill=False,
                               lw=2))

        variance_x = self.covariance_target[0,0]
        variance_y = self.covariance_target[1,1]
        range_variance1 = np.array([np.sqrt(variance_x),np.sqrt(variance_y)])
        variance_x = self.covariance_target[2,2]
        variance_y = self.covariance_target[3,3]
        range_variance2 = np.array([np.sqrt(variance_x),np.sqrt(variance_y)])
        p1 = self.target[:2] - range_variance1
        p2 = self.target[2:4] + range_variance2
        wh = p2 - p1


        #create simple line plot
        ax.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

        #add rectangle to plot

        ax.add_patch(Rectangle(p1.tolist(), wh[0], wh[1],
                               edgecolor = color,
                               fill=False,
                               linestyle ='--',
                               lw=2))
tracker = Tracker(detections_poses[0,:],kalman)
id_frame = 0

In [ ]:
id_frame += 1
tracks,img,dets = dataset.get_frame_and_detections(id_frame,draw_detection=True)
# plt.imshow(img)
# plt.show()
tracker.process_frame(dets)
tracker.plot_tracking_position(img)


In [ ]:
tracker.target

# Tracking cherries

In [ ]:

bytetrack_params = """
tracker_type: bytetrack # (str) Tracker backend: botsort|bytetrack; choose bytetrack for the classic baseline
track_high_thresh: 0.25 # (float) First-stage match threshold; raise for cleaner tracks, lower to keep more
track_low_thresh: 0.1 # (float) Second-stage threshold for low-score matches; balances recovery vs drift
new_track_thresh: 0.25 # (float) Start a new track if no match ≥ this; higher reduces false tracks
track_buffer: 30 # (int) Frames to keep lost tracks alive; higher handles occlusion, increases ID switches risk
match_thresh: 0.8 # (float) Association similarity threshold (IoU/cost); tune with detector quality
fuse_score: True # (bool) Fuse detection score with motion/IoU for matching; stabilizes weak detections
"""

botsort_params = """
tracker_type: botsort # (str) Tracker backend: botsort|bytetrack; choose botsort to enable BoT-SORT features
track_high_thresh: 0.25 # (float) First-stage match threshold; raise for cleaner tracks, lower to keep more
track_low_thresh: 0.1 # (float) Second-stage threshold for low-score matches; balances recovery vs drift
new_track_thresh: 0.25 # (float) Start a new track if no match ≥ this; higher reduces false tracks
track_buffer: 30 # (int) Frames to keep lost tracks alive; higher handles occlusion, increases ID switches risk
match_thresh: 0.8 # (float) Association similarity threshold (IoU/cost); tune with detector quality
fuse_score: True # (bool) Fuse detection score with motion/IoU for matching; stabilizes weak detections

# BoT-SORT specifics
gmc_method: sparseOptFlow # (str) Global motion compensation: sparseOptFlow|orb|none; helps moving camera scenes

# ReID model related thresh
proximity_thresh: 0.5 # (float) Min IoU to consider tracks proximate for ReID; higher is stricter
appearance_thresh: 0.8 # (float) Min appearance similarity for ReID; raise to avoid identity swaps
with_reid: False # (bool) Enable ReID model use; needs extra model and compute
model: auto # (str) ReID model name/path; "auto" uses detector features if available
"""

bytetrack_dict = yaml.safe_load(bytetrack_params)
botsort_dict = yaml.safe_load(botsort_params)
with open('bytetrack.yaml', 'w') as file:
    yaml.dump(bytetrack_dict, file)

with open('botsort.yaml', 'w') as file:
    yaml.dump(botsort_dict, file)

# Video Tracking
1. Generate a video with tracked cherries and analyze the results.

In [ ]:
model = YOLO('yolov11_best2.pt')
path_video = './short_video2.mov'
results = model.track(path_video, conf=0.3, iou=0.5, save=True, show=True, tracker="bytetrack.yaml")

In [ ]:
results

# Count and classify
1. Count how many cherries are across the video

In [ ]:
cap = cv2.VideoCapture(path_video)
success = True
# Loop through the video frames
while success:
# Read a frame from the video
  success, frame = cap.read()

  if success:
    results = model.track([frame[:shape[1],:shape[1]]], persist=True)
    ### CODE HERE